In [ ]:
# Logs
# [('userid', 'sessionid', 'timestamp'), 'action', 'value', 'category', 'age', 'gender', 'city']

# Sessions
# [('userid', 'sessionid'), 'начало', 'продолжительность', 'кол-во покупок', 'общ сумма', {'category', 'age', 'gender', 'city'}]

# Users
# [('userid'), 'age', 'gender', 'city', 'кол-во сессий', 'как давно начал', 'сумма времени', 'последний заход', 'вся сумма [по категориям]']

# [Cities] & [Categories] по желанию

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

from config import PATH, get_df, save_df

df = get_df()


def age_to_bin(age: int) -> int:
    if age <= 15:
        return 1
    if age <= 24:
        return 2
    if age <= 35:
        return 3
    if age <= 48:
        return 4
    if age <= 60:
        return 5
    else:
        return 6

def age_to_bin_name(age: int) -> str:
    if age <= 15:
        return "<16"
    if age <= 24:
        return "16-24"
    if age <= 35:
        return "25-35"
    if age <= 48:
        return "36-48"
    if age <= 60:
        return "48-60"
    else:
        return ">60"

<class 'pandas.core.frame.DataFrame'>
Index: 7510522 entries, 0 to 7510521
Data columns (total 9 columns):
 #   Column     Dtype         
---  ------     -----         
 0   userid     int64         
 1   sessionid  int64         
 2   timestamp  datetime64[ns]
 3   action     object        
 4   value      float64       
 5   category   object        
 6   age        int64         
 7   gender     object        
 8   city       object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 573.0+ MB
None


,userid,sessionid,timestamp,action,value,category,age,gender,city
0,10000,1,2021-12-30 12:28:45,category,0.0,Товары для детей,42,Ж,Красноярск
1,10000,1,2021-12-30 12:29:10,search,0.0,Товары для детей,42,Ж,Красноярск
2,10000,1,2021-12-30 12:29:25,product,0.0,Товары для детей,42,Ж,Красноярск
3,10000,1,2021-12-30 12:29:26,search,0.0,Товары для детей,42,Ж,Красноярск
4,10000,1,2021-12-30 12:29:34,search,0.0,Товары для детей,42,Ж,Красноярск
...,...,...,...,...,...,...,...,...,...
95,10000,14,2022-01-23 12:54:29,product,0.0,Электроника и бытовая техника,42,Ж,Красноярск
96,10000,14,2022-01-23 12:54:56,category,0.0,Электроника и бытовая техника,42,Ж,Красноярск
97,10000,15,2022-01-24 12:55:03,search,0.0,Электроника и бытовая техника,42,Ж,Красноярск
98,10000,15,2022-01-24 12:55:23,product,0.0,Электроника и бытовая техника,42,Ж,Красноярск


In [2]:
print(df.nunique(), end='\n_'*4+'\n')
for name in ['action', 'city', 'gender', 'category']:  # 'age'
    print(df[name].value_counts())
    
print('\n_'*4)
for name in df:
    print(name, len(df[df[name].isna()]))

userid          2500
sessionid        586
timestamp    7168835
action             7
value         170213
category           6
age               75
gender             2
city              14
dtype: int64
_
_
_
_
action
search          2094744
product         2012382
category        1595004
mainpage         823706
cart             696711
checkout         191520
confirmation      96455
Name: count, dtype: int64
city
Москва             2104849
Санкт-Петербург     948642
Новосибирск         575670
Екатеринбург        543015
Нижний Новгород     428970
Казань              395626
Красноярск          352194
Волгоград           333001
Ростов-на-Дону      330872
Самара              324321
Челябинск           297719
Воронеж             294179
Омск                292162
Уфа                 289302
Name: count, dtype: int64
gender
Ж    4342998
М    3167524
Name: count, dtype: int64
category
Товары для детей                 1172161
Красота и здоровье               1109183
Одежда и обувь                

In [3]:
cats = sorted([x for x in df['category'].unique() if x == x])
cats

['Авто/мото товары',
 'Красота и здоровье',
 'Одежда и обувь',
 'Продовольственные товары',
 'Товары для детей',
 'Электроника и бытовая техника']

In [ ]:
# myagg = {
#         'values_sum' : ('values', 'sum'),
#         'values_positive_count' : ('p', 'sum'),
#         'values_negative_count' : ('n', 'sum'),
#         'values_negative_sum' : ('n_sum','sum')
# }
# df_agg = (df.assign(p = df['values'].gt(0),
#                     n = df['values'].lt(0),
#                     n_sum = df['values'].lt(0).mul(df['values']),
#                    )
#             .groupby([pd.Grouper(freq='D'), 'types'])
#             .agg(**myagg))

In [12]:
def dataset_by_session(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['act_confirm'] = df['action'].eq('confirmation')
    df['act_check'] = df['action'].eq('checkout')
    df['cur_val'] = df['action'].eq('confirmation').mul(df['value'])
    
    res_df = df.groupby(["userid", "sessionid"]).agg(
        # Main
        userid=pd.NamedAgg(column="userid", aggfunc="first"),
        sessionid=pd.NamedAgg(column="sessionid", aggfunc="first"),
        age=pd.NamedAgg(column="age", aggfunc="first"),
        age_bins=pd.NamedAgg(column="age", aggfunc=lambda x: None),
        gender=pd.NamedAgg(column="gender", aggfunc="first"),
        city=pd.NamedAgg(column="city", aggfunc="first"),
        
        # Time
        time_start=pd.NamedAgg(column="timestamp", aggfunc="min"),
        time_end=pd.NamedAgg(column="timestamp", aggfunc="max"),
        time_full=pd.NamedAgg(column="timestamp", aggfunc=lambda x: None),
        time_full_s=pd.NamedAgg(column="timestamp", aggfunc=lambda x: None),
        # Value
        value=pd.NamedAgg(column="cur_val", aggfunc="sum"),
        # Other
        count_confirm=pd.NamedAgg(column="act_confirm", aggfunc="sum"),
        count_check=pd.NamedAgg(column="act_check", aggfunc="sum"),
        count_pages=pd.NamedAgg(column="sessionid", aggfunc="count"),
    ).reset_index(drop=True).sort_values(by=['userid', 'sessionid'], ignore_index=True)
    
    res_df['age_bins'] = res_df['age'].apply(lambda x: age_to_bin(x))
    
    res_df['time_full'] = res_df['time_end'] - res_df['time_start']
    res_df['time_full_s'] = res_df['time_full'].dt.total_seconds().astype('int64')

    # res_df.at[key[0], 'time_full_s'] = int(delta.total_seconds())
    # for key, sub_df in df.groupby(["userid", "sessionid"]):
    #     userid, sessionid = key
    #     res_df.at[key, 'count_confirm'] = sub_df.loc[sub_df['action'] == 'confirmation'].count()
    #     res_df.at[key, 'count_check'] = sub_df.loc[sub_df['action'] == 'checkout'].count()

    return res_df

In [13]:
sessions = dataset_by_session(df)

print(sessions.info())
display(sessions.head(10))
# display(sessions[sessions['count_confirm'] > 0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1399179 entries, 0 to 1399178
Data columns (total 14 columns):
 #   Column         Non-Null Count    Dtype          
---  ------         --------------    -----          
 0   userid         1399179 non-null  int64          
 1   sessionid      1399179 non-null  int64          
 2   age            1399179 non-null  int64          
 3   age_bins       1399179 non-null  int64          
 4   gender         1399179 non-null  object         
 5   city           1399179 non-null  object         
 6   time_start     1399179 non-null  datetime64[ns] 
 7   time_end       1399179 non-null  datetime64[ns] 
 8   time_full      1399179 non-null  timedelta64[ns]
 9   time_full_s    1399179 non-null  int64          
 10  value          1399179 non-null  float64        
 11  count_confirm  1399179 non-null  int64          
 12  count_check    1399179 non-null  int64          
 13  count_pages    1399179 non-null  int64          
dtypes: datetime64[ns](

,userid,sessionid,age,age_bins,gender,city,time_start,time_end,time_full,time_full_s,value,count_confirm,count_check,count_pages
0,10000,1,42,4,Ж,Красноярск,2021-12-30 12:28:45,2021-12-30 12:30:52,0 days 00:02:07,127,0.0,0,1,10
1,10000,2,42,4,Ж,Красноярск,2022-01-04 12:31:01,2022-01-04 12:31:01,0 days 00:00:00,0,0.0,0,0,1
2,10000,3,42,4,Ж,Красноярск,2022-01-04 12:31:26,2022-01-04 12:37:08,0 days 00:05:42,342,0.0,0,0,21
3,10000,4,42,4,Ж,Красноярск,2022-01-05 12:37:49,2022-01-05 12:37:49,0 days 00:00:00,0,0.0,0,0,1
4,10000,5,42,4,Ж,Красноярск,2022-01-05 12:38:24,2022-01-05 12:39:08,0 days 00:00:44,44,0.0,0,0,4
5,10000,6,42,4,Ж,Красноярск,2022-01-06 12:39:31,2022-01-06 12:41:30,0 days 00:01:59,119,0.0,0,0,7
6,10000,7,42,4,Ж,Красноярск,2022-01-09 12:41:38,2022-01-09 12:41:50,0 days 00:00:12,12,0.0,0,0,2
7,10000,8,42,4,Ж,Красноярск,2022-01-10 12:41:58,2022-01-10 12:44:36,0 days 00:02:38,158,11150.0,1,1,12
8,10000,9,42,4,Ж,Красноярск,2022-01-12 12:44:53,2022-01-12 12:45:24,0 days 00:00:31,31,0.0,0,0,3
9,10000,10,42,4,Ж,Красноярск,2022-01-13 12:45:44,2022-01-13 12:46:51,0 days 00:01:07,67,0.0,0,0,4


In [9]:
def dataset_by_user(df: pd.DataFrame, dfsess: pd.DataFrame) -> pd.DataFrame:
    res_df = df.groupby(["userid"]).agg(
        # Main
        userid=pd.NamedAgg(column="userid", aggfunc="first"),
        age=pd.NamedAgg(column="age", aggfunc=pd.Series.mode),
        gender=pd.NamedAgg(column="gender", aggfunc=pd.Series.mode),
        city=pd.NamedAgg(column="city", aggfunc=pd.Series.mode),
        count_sessions=pd.NamedAgg(column="sessionid", aggfunc="nunique"),
        # Time
        time_first=pd.NamedAgg(column="timestamp", aggfunc="min"),
        time_last=pd.NamedAgg(column="timestamp", aggfunc="max"),
        time_last_purchase=pd.NamedAgg(column="timestamp", aggfunc="max"),
        time_full=pd.NamedAgg(column="timestamp", aggfunc=lambda x: None),
        time_full_s=pd.NamedAgg(column="timestamp", aggfunc=lambda x: None),
        # Value
        value=pd.NamedAgg(column="value", aggfunc=lambda x: 0.),
        value_c1=pd.NamedAgg(column="value", aggfunc=lambda x: 0.),
        value_c2=pd.NamedAgg(column="value", aggfunc=lambda x: 0.),
        value_c3=pd.NamedAgg(column="value", aggfunc=lambda x: 0.),
        value_c4=pd.NamedAgg(column="value", aggfunc=lambda x: 0.),
        value_c5=pd.NamedAgg(column="value", aggfunc=lambda x: 0.),
        value_c6=pd.NamedAgg(column="value", aggfunc=lambda x: 0.),
        # Another
        # age_bins=pd.NamedAgg(column="age", aggfunc=age_to_bin),
        # age_bins_name=pd.NamedAgg(column="age", aggfunc=age_to_bin_name),
    )
    
    res_df['age_bins'] = res_df['age'].apply(lambda x: age_to_bin(x))
    res_df['age_bins_name'] = res_df['age'].apply(lambda x: age_to_bin_name(x))
    
    cats = sorted([x for x in df['category'].unique() if x == x])
    print(cats)
    for key, sub_df in df.groupby(["userid"]):
        userid = key[0] # print(userid, type(sub_df), len(sub_df))
        res_df.at[userid, 'time_last_purchase'] = sub_df.loc[sub_df['action'] == 'confirmation']['timestamp'].max()
        res_df.at[userid, 'value'] = sub_df.loc[sub_df['action'] == 'confirmation']['value'].sum()
        for i, cat in enumerate(cats):
            res_df.at[userid, f'value_c{i+1}'] = (
                sub_df.loc[(sub_df['action'] == 'cart') & (sub_df['category'] == cat)]['value'].sum())
    
    for key, sub_df in dfsess.groupby(["userid"]):
        delta = sub_df['time_full'].sum()
        res_df.at[key[0], 'time_full'] = delta
        res_df.at[key[0], 'time_full_s'] = int(delta.total_seconds())
    # res_df['time_full'] = res_df['time_full'].astype(pd.Timedelta)
    # res_df['time_full_s'] = res_df['time_full'].dt.total_seconds()#.round()
    
    return res_df.reset_index(drop=True).sort_values(by='userid', ignore_index=True)

In [10]:
users = dataset_by_user(df, sessions)

print(users.info())
display(users.head(10))

['Авто/мото товары', 'Красота и здоровье', 'Одежда и обувь', 'Продовольственные товары', 'Товары для детей', 'Электроника и бытовая техника']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   userid              2500 non-null   int64         
 1   age                 2500 non-null   int64         
 2   gender              2500 non-null   object        
 3   city                2500 non-null   object        
 4   count_sessions      2500 non-null   int64         
 5   time_first          2500 non-null   datetime64[ns]
 6   time_last           2500 non-null   datetime64[ns]
 7   time_last_purchase  2500 non-null   datetime64[ns]
 8   time_full           2500 non-null   object        
 9   time_full_s         2500 non-null   object        
 10  value               2500 non-null   float64       
 11  value_c1          

,userid,age,gender,city,count_sessions,time_first,time_last,time_last_purchase,time_full,time_full_s,value,value_c1,value_c2,value_c3,value_c4,value_c5,value_c6,age_bins,age_bins_name
0,10000,42,Ж,Красноярск,559,2021-12-30 12:28:45,2025-01-03 00:31:55,2024-11-15 23:56:50,0 days 09:41:10,34870,398622.32,8.014473e+05,8.706818e+05,8.667981e+05,6.232833e+05,8.154801e+05,9.197128e+05,4,36-48
1,10001,51,Ж,Омск,555,2021-12-30 03:21:27,2025-01-02 16:06:14,2024-12-18 15:57:54,0 days 10:32:26,37946,408376.03,1.013801e+06,6.408361e+05,8.831472e+05,9.496096e+05,1.089472e+06,6.740919e+05,5,48-60
2,10002,25,М,Санкт-Петербург,562,2021-12-28 08:25:21,2025-01-02 20:08:25,2024-12-25 20:02:52,0 days 09:30:10,34210,226000.78,7.070012e+05,8.811871e+05,5.996066e+05,5.226358e+05,8.429284e+05,9.467293e+05,3,25-35
3,10003,34,М,Волгоград,550,2021-12-28 19:09:17,2025-01-01 07:13:57,2024-12-13 07:03:21,0 days 09:51:52,35512,236511.00,1.010398e+06,8.642704e+05,7.395079e+05,7.739072e+05,1.041259e+06,9.039430e+05,3,25-35
4,10004,29,Ж,Москва,585,2021-12-26 23:21:19,2025-01-03 12:20:48,2024-12-31 12:19:44,0 days 10:26:41,37601,214624.00,6.315062e+05,1.023260e+06,6.062747e+05,1.238431e+06,1.051989e+06,8.185748e+05,3,25-35
5,10005,18,М,Нижний Новгород,534,2021-12-28 14:22:45,2025-01-02 02:56:32,2024-12-26 02:51:00,0 days 10:27:25,37645,236027.00,1.128231e+06,7.566880e+05,1.037747e+06,1.101959e+06,7.342136e+05,5.635971e+05,2,16-24
6,10006,25,Ж,Самара,575,2021-12-28 14:23:09,2025-01-03 03:29:34,2025-01-03 03:29:34,0 days 10:48:11,38891,206972.49,5.999327e+05,1.011365e+06,1.117357e+06,7.939139e+05,6.345159e+05,1.199847e+06,3,25-35
7,10007,42,Ж,Москва,563,2021-12-28 00:05:25,2025-01-01 13:12:58,2024-10-11 12:02:57,0 days 10:41:36,38496,289888.52,1.004222e+06,7.193297e+05,1.133279e+06,7.815493e+05,9.677546e+05,1.189693e+06,4,36-48
8,10008,35,М,Москва,559,2021-12-30 08:57:28,2024-12-31 21:47:20,2024-12-29 21:46:46,0 days 10:31:05,37865,255078.00,8.847339e+05,1.224847e+06,6.534206e+05,9.756731e+05,9.305712e+05,6.819890e+05,3,25-35
9,10009,18,Ж,Москва,555,2021-12-30 07:37:10,2025-01-01 20:24:59,2024-12-08 20:04:42,0 days 10:38:39,38319,167668.86,4.449358e+05,1.017976e+06,9.456638e+05,7.108812e+05,1.224532e+06,8.991400e+05,2,16-24


In [14]:
# save_df(df, 'logs.csv')
save_df(sessions, 'sessions.csv')
# save_df(users, 'users.csv')